In [1]:
!pip install llama-index langchain_community "httpx==0.27.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Su

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk"


In [3]:
from llama_index.core.tools import FunctionTool

# 주피터 노트북 환경에서 비동기 코드를 실행할 때 발생하는 이벤트 루프 중첩 문제를 방지
# 비동기 코드란 작업을 병렬로 처리할 수 있는 방법을 의미합니다
import nest_asyncio
nest_asyncio.apply()

def multiply(a: int, b: int) -> int:
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply) # multiply 함수를 곱하기 도구로 등록

def divide(a: int, b: int) -> int:
    return a / b

divide_tool = FunctionTool.from_defaults(fn=divide) # divide 함수를 나누기 도구로 등록
tools = [multiply_tool, divide_tool] # 등록된 도구들을 리스트로 관리

In [4]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
agent_engine = OpenAIAgentWorker.from_tools(tools, llm=llm, verbose=True) # 도구와 LLM을 결합하여 에이전트를 생성
agent = AgentRunner(agent_engine)

task = agent.create_task("(121 * 12) / 4 결과는?") # "(121 * 12) / 4? 결과는?"을 작업으로 생성

# 모든 단계 실행
while len(agent.get_upcoming_steps(task.task_id)) > 0: # 작업 ID를 기반으로 아직 실행되지 않은 단계(step)들을 반환
    agent.run_step(task.task_id) # 한 번에 하나의 단계를 실행

# 최종 응답 확인
response = agent.finalize_response(task.task_id) # 모든 단계가 완료되었는지 확인한 후, 작업의 최종 결과를 생성
print(f"Final Response: {response}")

Added user message to memory: (121 * 12) / 4 결과는?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 12}
Got output: 1452

=== Calling Function ===
Calling function: divide with args: {"a": 1452, "b": 4}
Got output: 363.0

Final Response: \( (121 \times 12) \div 4 \)의 결과는 363입니다.
